## Predicting Income Hackathon
# EDA and Cleaning

### Team: Ingrid Wang and Jena Brentano

In [176]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

### Read in Data and Explore

In [177]:
df = pd.read_csv('./data/large_train_sample.csv')
df_test = pd.read_csv('./data/test_data.csv')

In [178]:
df_test.shape

(16281, 13)

In [179]:
df.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,wage
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,Male,0,0,40,United-States,<=50K


In [180]:
# Rename Columns
df.columns = df.columns.str.lower().str.replace(' ', '')
df.columns = df.columns.str.lower().str.replace('-', '_')
df_test.columns = df_test.columns.str.lower().str.replace(' ', '')
df_test.columns = df_test.columns.str.lower().str.replace('-', '_')

In [181]:
# Check for Null Values
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
wage              0
dtype: int64

In [182]:
df.describe()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [183]:
df.head(2)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,wage
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,13,United-States,<=50K


#### Clean up data - get rid of question marks in columns"

In [184]:
df_test['workclass'].value_counts()

 Private             11210
 Self-emp-not-inc     1321
 Local-gov            1043
 ?                     963
 State-gov             683
 Self-emp-inc          579
 Federal-gov           472
 Without-pay             7
 Never-worked            3
Name: workclass, dtype: int64

In [185]:
df_test['workclass'].replace(' ?', 'Private', inplace=True)

In [186]:
df_test['occupation'].value_counts()

 Prof-specialty       2032
 Exec-managerial      2020
 Craft-repair         2013
 Sales                1854
 Adm-clerical         1841
 Other-service        1628
 Machine-op-inspct    1020
 ?                     966
 Transport-moving      758
 Handlers-cleaners     702
 Tech-support          518
 Farming-fishing       496
 Protective-serv       334
 Priv-house-serv        93
 Armed-Forces            6
Name: occupation, dtype: int64

In [187]:
df_test['occupation'].replace(' ?', 'Other-service', inplace=True)

In [188]:
df[df['workclass'].str.contains("\?")]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,wage
27,54,?,180211,Some-college,10,Married-civ-spouse,?,Husband,Male,0,0,60,South,>50K
61,32,?,293936,7th-8th,4,Married-spouse-absent,?,Not-in-family,Male,0,0,40,?,<=50K
69,25,?,200681,Some-college,10,Never-married,?,Own-child,Male,0,0,40,United-States,<=50K
77,67,?,212759,10th,6,Married-civ-spouse,?,Husband,Male,0,0,2,United-States,<=50K
106,17,?,304873,10th,6,Never-married,?,Own-child,Female,34095,0,32,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32530,35,?,320084,Bachelors,13,Married-civ-spouse,?,Wife,Female,0,0,55,United-States,>50K
32531,30,?,33811,Bachelors,13,Never-married,?,Not-in-family,Female,0,0,99,United-States,<=50K
32539,71,?,287372,Doctorate,16,Married-civ-spouse,?,Husband,Male,0,0,10,United-States,>50K
32541,41,?,202822,HS-grad,9,Separated,?,Not-in-family,Female,0,0,32,United-States,<=50K


In [189]:
df.shape

(32561, 14)

### Create Classes, 1 for salary > 50k, 0 for salary <= 50k

In [191]:
df['wage'] = df['wage'].map({' <=50K':0, ' >50K':1})

In [192]:
df['workclass'].value_counts()

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64

In [193]:
df['marital_status'].value_counts()

 Married-civ-spouse       14976
 Never-married            10683
 Divorced                  4443
 Separated                 1025
 Widowed                    993
 Married-spouse-absent      418
 Married-AF-spouse           23
Name: marital_status, dtype: int64

### Feature Engineering

#### Get Dummies for non-numerical columns

In [194]:
dummies = ['workclass',
           'marital_status',
           'relationship',
           'occupation',
           'sex',
           'native_country']

In [195]:
df = pd.get_dummies(df, columns = dummies, drop_first=True)
df_test = pd.get_dummies(df_test, columns = dummies, drop_first=True)

In [196]:
df.head(3)

,age,fnlwgt,education,education_num,capital_gain,capital_loss,hours_per_week,wage,workclass_ Federal-gov,workclass_ Local-gov,...,native_country_ Portugal,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia
0,39,77516,Bachelors,13,2174,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,Bachelors,13,0,0,13,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,HS-grad,9,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [197]:
df.columns

Index(['age', 'fnlwgt', 'education', 'education_num', 'capital_gain',
       'capital_loss', 'hours_per_week', 'wage', 'workclass_ Federal-gov',
       'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private',
       'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc',
       'workclass_ State-gov', 'workclass_ Without-pay',
       'marital_status_ Married-AF-spouse',
       'marital_status_ Married-civ-spouse',
       'marital_status_ Married-spouse-absent',
       'marital_status_ Never-married', 'marital_status_ Separated',
       'marital_status_ Widowed', 'relationship_ Not-in-family',
       'relationship_ Other-relative', 'relationship_ Own-child',
       'relationship_ Unmarried', 'relationship_ Wife',
       'occupation_ Adm-clerical', 'occupation_ Armed-Forces',
       'occupation_ Craft-repair', 'occupation_ Exec-managerial',
       'occupation_ Farming-fishing', 'occupation_ Handlers-cleaners',
       'occupation_ Machine-op-inspct', 'occupation_ Other-s

In [198]:
# Rename new dummy columns
df.columns = df.columns.str.lower().str.replace(' ', '')
df_test.columns = df_test.columns.str.lower().str.replace(' ', '')

#### Simplify and Narrow down Features

In [199]:
# Simplify number of Education categories to 8 from 16
df_test['education_num'] = df_test['education_num'].map({1:8,2:8,3:8,4:8,
                                               5:8,6:8,7:8, 8:8,
                                               9:9, 10:10, 11:11,
                                               12:12, 13:13, 14:14,
                                              15:15, 16:16})
df['education_num'] = df['education_num'].map({1:8,2:8,3:8,4:8,
                                               5:8,6:8,7:8, 8:8,
                                               9:9, 10:10, 11:11,
                                               12:12, 13:13, 14:14,
                                              15:15, 16:16})

In [200]:
# Create one column that combines capital gains and capital loss into capital net
df['capital_gains_net'] = df['capital_gain'] - df['capital_loss']
df_test['capital_gains_net'] = df_test['capital_gain'] - df_test['capital_loss']

df.drop(columns=['capital_gain','capital_loss'], inplace=True)
df_test.drop(columns=['capital_gain','capital_loss'], inplace=True)

In [201]:
# Check out balance of classes
df['wage'].value_counts(normalize=True)

0    0.75919
1    0.24081
Name: wage, dtype: float64

In [202]:
df.columns.to_list()

['age',
 'fnlwgt',
 'education',
 'education_num',
 'hours_per_week',
 'wage',
 'workclass_federal-gov',
 'workclass_local-gov',
 'workclass_never-worked',
 'workclass_private',
 'workclass_self-emp-inc',
 'workclass_self-emp-not-inc',
 'workclass_state-gov',
 'workclass_without-pay',
 'marital_status_married-af-spouse',
 'marital_status_married-civ-spouse',
 'marital_status_married-spouse-absent',
 'marital_status_never-married',
 'marital_status_separated',
 'marital_status_widowed',
 'relationship_not-in-family',
 'relationship_other-relative',
 'relationship_own-child',
 'relationship_unmarried',
 'relationship_wife',
 'occupation_adm-clerical',
 'occupation_armed-forces',
 'occupation_craft-repair',
 'occupation_exec-managerial',
 'occupation_farming-fishing',
 'occupation_handlers-cleaners',
 'occupation_machine-op-inspct',
 'occupation_other-service',
 'occupation_priv-house-serv',
 'occupation_prof-specialty',
 'occupation_protective-serv',
 'occupation_sales',
 'occupation_tec

In [203]:
# Keep, but don't run again:
df.to_csv('./data/clean_df.csv', index=False)

In [204]:
# Keep, but don't run again:
df_test.to_csv('./data/clean_test.csv', index=False)

In [205]:
df.columns.to_list()

['age',
 'fnlwgt',
 'education',
 'education_num',
 'hours_per_week',
 'wage',
 'workclass_federal-gov',
 'workclass_local-gov',
 'workclass_never-worked',
 'workclass_private',
 'workclass_self-emp-inc',
 'workclass_self-emp-not-inc',
 'workclass_state-gov',
 'workclass_without-pay',
 'marital_status_married-af-spouse',
 'marital_status_married-civ-spouse',
 'marital_status_married-spouse-absent',
 'marital_status_never-married',
 'marital_status_separated',
 'marital_status_widowed',
 'relationship_not-in-family',
 'relationship_other-relative',
 'relationship_own-child',
 'relationship_unmarried',
 'relationship_wife',
 'occupation_adm-clerical',
 'occupation_armed-forces',
 'occupation_craft-repair',
 'occupation_exec-managerial',
 'occupation_farming-fishing',
 'occupation_handlers-cleaners',
 'occupation_machine-op-inspct',
 'occupation_other-service',
 'occupation_priv-house-serv',
 'occupation_prof-specialty',
 'occupation_protective-serv',
 'occupation_sales',
 'occupation_tec

In [171]:
df = pd.read_csv('./data/clean_df.csv')

In [172]:
df.head(2)

,age,fnlwgt,education,education_num,hours_per_week,wage,workclass_federal-gov,workclass_local-gov,workclass_never-worked,workclass_private,...,native_country_puerto-rico,native_country_scotland,native_country_south,native_country_taiwan,native_country_thailand,native_country_trinadad&tobago,native_country_united-states,native_country_vietnam,native_country_yugoslavia,capital_gains_net
0,39,77516,Bachelors,13,40,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,2174
1,50,83311,Bachelors,13,13,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
